In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Fitness_trackers_updated.csv')

In [3]:
df.head()

,Brand Name,Device Type,Model Name,Color,Selling Price,Original Price,Display,Rating (Out of 5),Strap Material,Average Battery Life (in days),Reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,"2,499","2,999",AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,"2,099","2,499",AMOLED Display,4.2,Thermoplastic polyurethane,14,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,"1,722","2,099",LCD Display,3.5,Leather,14,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,"2,469","2,999",AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
4,Xiaomi,FitnessBand,Band 3,Black,"1,799","2,199",OLED Display,4.3,Plastic,7,NaN


## Clean column names

In [4]:
df = df.rename(columns = {'Rating (Out of 5)' : 'rating', 'Average Battery Life (in days)' : 'avg_battery_life_days'})

In [5]:
df.columns

Index(['Brand Name', 'Device Type', 'Model Name', 'Color', 'Selling Price',
       'Original Price', 'Display', 'rating', 'Strap Material',
       'avg_battery_life_days', 'Reviews'],
      dtype='object')

In [6]:
new_cols = [i.lower().replace(' ', '_') for i in df.columns]

Add INR (Indian Rupees) to prices:

In [7]:
new_cols = [i + '_INR' if i.endswith('price') else i for i in new_cols]

In [8]:
new_cols

['brand_name',
 'device_type',
 'model_name',
 'color',
 'selling_price_INR',
 'original_price_INR',
 'display',
 'rating',
 'strap_material',
 'avg_battery_life_days',
 'reviews']

In [9]:
df.columns = new_cols

## Fix dtypes

In [10]:
df.dtypes

brand_name                object
device_type               object
model_name                object
color                     object
selling_price_INR         object
original_price_INR        object
display                   object
rating                   float64
strap_material            object
avg_battery_life_days      int64
reviews                   object
dtype: object

At least, to start with, `selling_price` and `original_price` should be `float` dtype.

In [11]:
df.selling_price_INR = df.selling_price_INR.str.replace(',', '').astype(float)
df.original_price_INR = df.original_price_INR.str.replace(',', '').astype(float)

Create additional price columns in USD. Current exchange rate (2023-10-09) is 1000 INR = 12 USD

In [13]:
df['selling_price_USD'] = df.selling_price_INR / 1000 * 12
df['original_price_USD'] = df.original_price_INR / 1000 * 12

In [14]:
df.dtypes

brand_name                object
device_type               object
model_name                object
color                     object
selling_price_INR        float64
original_price_INR       float64
display                   object
rating                   float64
strap_material            object
avg_battery_life_days      int64
reviews                   object
selling_price_USD        float64
original_price_USD       float64
dtype: object

Reorder the columns:

In [15]:
df.columns

Index(['brand_name', 'device_type', 'model_name', 'color', 'selling_price_INR',
       'original_price_INR', 'display', 'rating', 'strap_material',
       'avg_battery_life_days', 'reviews', 'selling_price_USD',
       'original_price_USD'],
      dtype='object')

In [17]:
rearranged_cols = ['brand_name', 'device_type', 'model_name', 'color',
                   'selling_price_INR', 'selling_price_USD',
                   'original_price_INR', 'original_price_USD',
                   'display', 'rating', 'strap_material',
                   'avg_battery_life_days', 'reviews']

In [19]:
df = df[rearranged_cols]

In [20]:
df.head(2)

,brand_name,device_type,model_name,color,selling_price_INR,selling_price_USD,original_price_INR,original_price_USD,display,rating,strap_material,avg_battery_life_days,reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,2499.0,29.988,2999.0,35.988,AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,2099.0,25.188,2499.0,29.988,AMOLED Display,4.2,Thermoplastic polyurethane,14,NaN


## Check object variables

In [21]:
df.select_dtypes(object)

,brand_name,device_type,model_name,color,display,strap_material,reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,AMOLED Display,Thermoplastic polyurethane,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,AMOLED Display,Thermoplastic polyurethane,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,LCD Display,Leather,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,AMOLED Display,Thermoplastic polyurethane,NaN
4,Xiaomi,FitnessBand,Band 3,Black,OLED Display,Plastic,NaN
...,...,...,...,...,...,...,...
605,Fire-Boltt,Smartwatch,Talk Bluetooth,"Black, Grey, Green",HD Display,Silicone,923
606,Fire-Boltt,Smartwatch,Almighty,Orange,AMOLED Display,Silicone,212
607,Fire-Boltt,Smartwatch,Mercury,Red,HD Display,Silicone,4941
608,Fire-Boltt,Smartwatch,Spin,White,TFT-LCD Display,Silicone,15


Reviews values seem to be the number of reviews that a particular device has, but the dtype is wrong.

In [87]:
df.reviews[df.reviews.notnull()]

7          2
8          3
73      6568
74     17809
75     23426
       ...  
605      923
606      212
607     4941
608       15
609      688
Name: reviews, Length: 114, dtype: object

In [89]:
#df.reviews.astype(float)

When attemping to convert to float, an error rose because some values are numbers with thousands comma. It will remove the thousands separator.

In [92]:
df.reviews = df.reviews.str.replace(',', '').astype(float)

Turn `color` and `strap_material` into categorical variables.

In [23]:
df.strap_material.unique()

array(['Thermoplastic polyurethane', 'Leather', 'Plastic', 'Silicone',
       'Elastomer', 'Rubber', 'Fabric', 'Nylon', 'Stainless Steel',
       'leather', 'Aluminium', 'Leather '], dtype=object)

In [28]:
df.strap_material = df.strap_material.astype('category')

In [47]:
df.color.unique()

array(['Black', 'Blue', 'Dual Color', 'Grey, Pink, Black',
       'Black, Pink, Beige', 'Black, Blue, Pink',
       'Storm Blue, Black, Rosewood', 'Maroon ',
       'Black, Desert Rose, Lunar White', 'White ', 'Granite Reflective ',
       'Lilac ', 'Violet, black, Grey, Teal, Blue, Red', 'Black ',
       'Grey ', 'Charcoal ', 'Multicolor ', 'Blue ', 'Purple',
       'Purple, White', 'Black, Plum, Grey', 'Plum silver', 'Purple ',
       'Orange', 'Pink ', 'Orange, Pink, Green',
       'Rose Gold, Gun Metal Black', 'Black, Purple, Teal, Blue',
       'Violet ', 'Light Blue', 'Sky Blue ', 'Yellow', 'Grey',
       'Pink, Black, Teal, Orange', 'Pink, Black',
       'Sandstone Grey, Meteorite Black, Coral Pink, ',
       'Meteorite Black, Midnight Navy', 'Dynamic Orange', 'Navy Blue',
       'Olive Green', 'Black, Blue, Red', 'Black, Blue', 'Black, Gold',
       'Pink, Silver, Black', 'Black, White', 'Black, Pink',
       'Purple, Black', 'Orange, Red, Black', 'Green, Grey',
       'Black, 

It turns out that the values of the `color` column needs some cleaning.  
First I will get a list of available colors for every row in the `color` column.

In [55]:
# create a copy to work with color values
colours = df.color.copy()

In [56]:
# remove trailing whitespaces and separate values into a list
colours = colours.str.strip().str.split(', ')
colours_titlecase = []
# apply transformation to every list of colours
for i in colours:
    # only 1 colour, just set case to title
    if len(i) == 1:
        colours_titlecase.append( i[0].title() )
    else :
        # several colours --> set case to title
        i = [colour.title() for colour in i]
        # create a single string with all the colours in the list
        i = ', '.join(i)
        colours_titlecase.append(i)
# create a series from colours_titlecase
colours = pd.Series(colours_titlecase)

In [58]:
colours.unique()

array(['Black', 'Blue', 'Dual Color', 'Grey, Pink, Black',
       'Black, Pink, Beige', 'Black, Blue, Pink',
       'Storm Blue, Black, Rosewood', 'Maroon',
       'Black, Desert Rose, Lunar White', 'White', 'Granite Reflective',
       'Lilac', 'Violet, Black, Grey, Teal, Blue, Red', 'Grey',
       'Charcoal', 'Multicolor', 'Purple', 'Purple, White',
       'Black, Plum, Grey', 'Plum Silver', 'Orange', 'Pink',
       'Orange, Pink, Green', 'Rose Gold, Gun Metal Black',
       'Black, Purple, Teal, Blue', 'Violet', 'Light Blue', 'Sky Blue',
       'Yellow', 'Pink, Black, Teal, Orange', 'Pink, Black',
       'Sandstone Grey, Meteorite Black, Coral Pink,',
       'Meteorite Black, Midnight Navy', 'Dynamic Orange', 'Navy Blue',
       'Olive Green', 'Black, Blue, Red', 'Black, Blue', 'Black, Gold',
       'Pink, Silver, Black', 'Black, White', 'Black, Pink',
       'Purple, Black', 'Orange, Red, Black', 'Green, Grey',
       'Black, Green', 'Black, Teal Green, Grey', 'Black, Red, Blue',
 

Finally, I will save the processed colours to the dataset

In [59]:
df.color = colours

In [61]:
df.select_dtypes(object)

,brand_name,device_type,model_name,color,display,reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,AMOLED Display,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,AMOLED Display,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,LCD Display,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,AMOLED Display,NaN
4,Xiaomi,FitnessBand,Band 3,Black,OLED Display,NaN
...,...,...,...,...,...,...
605,Fire-Boltt,Smartwatch,Talk Bluetooth,"Black, Grey, Green",HD Display,923
606,Fire-Boltt,Smartwatch,Almighty,Orange,AMOLED Display,212
607,Fire-Boltt,Smartwatch,Mercury,Red,HD Display,4941
608,Fire-Boltt,Smartwatch,Spin,White,TFT-LCD Display,15


### Check uniqueness of variables 

#### brand_name

In [62]:
df.brand_name.unique()

array(['Xiaomi', 'OnePlus ', 'FitBit', 'realme', 'Huawei', 'Honor',
       'GOQii', 'Infinix', 'LCARE', 'LAVA', 'Noise', 'Oppo', 'Fastrack',
       'boAt ', 'Noise ', 'huami', 'SAMSUNG ', 'FOSSIL ', 'APPLE',
       'GARMIN ', 'Crossbeats', 'dizo by realme ', 'Ptron', 'Zebronics',
       'Fire-Boltt'], dtype=object)

In [64]:
df.brand_name.str.title().unique()

array(['Xiaomi', 'Oneplus ', 'Fitbit', 'Realme', 'Huawei', 'Honor',
       'Goqii', 'Infinix', 'Lcare', 'Lava', 'Noise', 'Oppo', 'Fastrack',
       'Boat ', 'Noise ', 'Huami', 'Samsung ', 'Fossil ', 'Apple',
       'Garmin ', 'Crossbeats', 'Dizo By Realme ', 'Ptron', 'Zebronics',
       'Fire-Boltt'], dtype=object)

In [65]:
df.brand_name = df.brand_name.str.title()

#### device_type

In [66]:
df.device_type.unique()

array(['FitnessBand', 'Smartwatch'], dtype=object)

I will set the `device_type` to categorical.

In [67]:
df.device_type = df.device_type.astype('category')

#### model_name 

In [69]:
df.model_name.unique()

array(['Smart Band 5', 'Smart Band 4', 'HMSH01GE', 'Band 3',
       'Band - HRX Edition', 'Band 2', 'Revolve', 'RevolveActive',
       'Smart Band 3i', 'Steven Harrington Edition Band', 'Band',
       'Versa 2', 'Sense', 'Versa 3', 'Charge 4', 'Inspire', 'Inspire 2',
       'Lunar', 'FB413LVLV', 'Flex Small', 'Inspire HR', 'Charge 3',
       'Versa Special Edition', 'Ionic', 'Versa 2 Special Edition',
       'Versa', 'Surge', 'Charge 2', 'Charge 3 Special Edition', 'Blaze',
       'Alta HR', 'Versa Lite Edition', 'versa', 'Charge HR', 'Alta',
       'Smart band', 'Band 4', 'Band 6', 'Band 2 Pro Activity', 'Band 5i',
       'Band 5', 'band 4 running', 'Band Z1', 'Beat', 'Vital 3.0',
       'Watch', 'Mambo', 'BeFit', 'ColorFit', 'ColorFit2', '46 mm',
       '41mm', '2 Pro', '2S', 'S', 'S Pro', 'S Master', 'Fashion',
       'Classic', 'Reflex 3.0', 'Reflex Beat', 'Reflex Smart Band',
       'Reflex 2.0', 'Storm', 'Xplorer', 'Delta', 'O2',
       'ColorFit Qube SpO2', 'ColorFit Pro 3', 'Co

#### display

In [70]:
df.display.unique()

array(['AMOLED Display', 'LCD Display', 'OLED Display', 'PMOLED Display',
       'LED Display', 'TFT-LCD Display', 'OLED Retina Display',
       'IPS Display', 'HD Display'], dtype=object)

In [72]:
df.display = df.display.astype('category')

## Numerical variables

In [94]:
df.describe().round()

,selling_price_INR,selling_price_USD,original_price_INR,original_price_USD,rating,avg_battery_life_days,reviews
count,610.0,610.0,610.0,610.0,554.0,610.0,114.0
mean,20707.0,248.0,23978.0,288.0,4.0,9.0,1943.0
std,19804.0,238.0,20243.0,243.0,0.0,8.0,4771.0
min,799.0,10.0,1599.0,19.0,2.0,1.0,2.0
25%,6995.0,84.0,10249.0,123.0,4.0,2.0,80.0
50%,14999.0,180.0,18995.0,228.0,4.0,7.0,288.0
75%,27468.0,330.0,31417.0,377.0,4.0,14.0,904.0
max,122090.0,1465.0,122090.0,1465.0,5.0,45.0,23426.0


## Missing values

In [95]:
df.isna().sum()

brand_name                 0
device_type                0
model_name                 0
color                      0
selling_price_INR          0
selling_price_USD          0
original_price_INR         0
original_price_USD         0
display                    0
rating                    56
strap_material             0
avg_battery_life_days      0
reviews                  496
dtype: int64

It's common to have missing values in both `rating` and `reviews` columns. Not everybody gives a rating value or leaves a review.

I quick check to the rating values, which should be ranging from 0 to 5.

In [102]:
df.rating.agg([min, max, 'mean'])

min     2.000000
max     5.000000
mean    4.195668
Name: rating, dtype: float64

In [103]:
df.sample(5)

,brand_name,device_type,model_name,color,selling_price_INR,selling_price_USD,original_price_INR,original_price_USD,display,rating,strap_material,avg_battery_life_days,reviews
608,Fire-Boltt,Smartwatch,Spin,White,5846.0,70.152,5999.0,71.988,TFT-LCD Display,2.4,Silicone,5,15.0
428,Apple,Smartwatch,42 mm Space Gray Aluminium,Black Woven,29900.0,358.800,29900.0,358.800,OLED Retina Display,4.5,Aluminium,1,NaN
27,Fitbit,Smartwatch,Versa 2 Special Edition,Multicolor,15499.0,185.988,23999.0,287.988,AMOLED Display,4.4,Silicone,7,NaN
245,Fossil,Smartwatch,FTW5018 Hybrid,Gold,11596.0,139.152,14495.0,173.940,AMOLED Display,4.5,Stainless Steel,2,NaN
450,Garmin,Smartwatch,Forerunner 245 Music,"Black, Red",30000.0,360.000,36490.0,437.880,LED Display,4.2,Silicone,7,NaN


Finally, I will save the clean dataset:

In [ ]:
df.to_csv('clean_fitness_dataset.csv', index=False)